In [1]:
from vsp import *
import numpy as np
import vsp as vsp
from subprocess import call

In [2]:
def UpdateGeometry(actions):

    span = 1

    # OpenVSP Script Part
    stdout = vsp.cvar.cstdout
    errorMgr = vsp.ErrorMgrSingleton_getInstance()

    # WingBodyTestCase
    vsp.VSPRenew();
    errorMgr.PopErrorAndPrint(stdout)

    # Add Wing
    WingBody = vsp.AddGeom("WING", "")

    # Insert A Couple More Sections
    InsertXSec(WingBody, 1, XS_FOUR_SERIES);
    InsertXSec(WingBody, 1, XS_FOUR_SERIES);
    Update();

    # Cut The Original Section
    CutXSec(WingBody, 1);

    # Change Driver
    #SetDriverGroup(WingBody, 1, AREA_WSECT_DRIVER, ROOTC_WSECT_DRIVER, TIPC_WSECT_DRIVER);
    SetParmVal(WingBody, "RotateAirfoilMatchDideralFlag", "WingGeom", 1.0);



    #Work on blending
    SetParmVal(WingBody, "InLEMode", "XSec_0", 1);
    SetParmVal(WingBody, "OutLEMode", "XSec_0", 1);
    SetParmVal(WingBody, "InLEMode", "XSec_1", 1);
    SetParmVal(WingBody, "OutLEMode", "XSec_1", 1);
    SetParmVal(WingBody, "InLEMode", "XSec_2", 2);
    SetParmVal(WingBody, "OutLEMode", "XSec_2", 2);

    SetParmVal(WingBody, "InTEMode", "XSec_0", 1);
    SetParmVal(WingBody, "OutTEMode", "XSec_0", 1);
    SetParmVal(WingBody, "InTEMode", "XSec_1", 1);
    SetParmVal(WingBody, "OutTEMode", "XSec_1", 1);
    SetParmVal(WingBody, "InTEMode", "XSec_2", 5);
    SetParmVal(WingBody, "OutTEMode", "XSec_2", 5);
    Update();



    # Change Some Parameters 1st Section

    SetParmVal(WingBody, "Root_Chord", "XSec_1", 0.7);
    SetParmVal(WingBody, "Tip_Chord", "XSec_1", 0.3);
    SetParmVal(WingBody, "Span", "XSec_1", 0.25*span );
    SetParmVal(WingBody, "Sweep", "XSec_1", 35.0 );
    Update();

    # Change Some Parameters 2nd Section
    SetParmVal(WingBody, "Root_Chord", "XSec_2", 0.3);
    SetParmVal(WingBody, "Tip_Chord", "XSec_2", 0.1);
    SetParmVal(WingBody, "Span", "XSec_2", 0.75*span );
    SetParmVal(WingBody, "Sweep", "XSec_2", 35.0 );
    Update();

    #Use airfoil points
    for i in range(0,1):
        xsurf_id = vsp.GetXSecSurf(WingBody, i)
        ChangeXSecShape(xsurf_id, i, vsp.XS_FILE_AIRFOIL)
        xsec = vsp.GetXSec(xsurf_id, i)
        ReadFileAirfoil(xsec,"/home/simonx/Documents/Udacity/Projects/capstone/PESM_NSGA2_OpenVSP/Archive/000001.dat"); #SG6043
        Update();


    # File basename
    baseName = "WingBody";
    csvName = baseName + "_DegenGeom.csv";
    meshstlName = baseName + "_DegenGeom.stl";
    vspName = baseName + ".vsp3";
    stlName = baseName + ".stl";

    # Set File Name
    SetComputationFileName(DEGEN_GEOM_CSV_TYPE, csvName);
    SetComputationFileName(CFD_STL_TYPE, meshstlName);

    WriteVSPFile(vspName)
    ExportFile(stlName, SET_ALL, EXPORT_STL)
    ComputeDegenGeom( SET_ALL, DEGEN_GEOM_CSV_TYPE );
    ComputeCFDMesh( SET_ALL, CFD_STL_TYPE );        # Mesh

    # Check for errors

    num_err = errorMgr.GetNumTotalErrors()
    for i in range(0, num_err):
        err = errorMgr.PopLastError()
        print "error = ", err.m_ErrorString


In [3]:
#These numbers are not connected at the moment
actions = [1,2,3,4,5,6,7,8]
#After running you will have some new geometry and mesh documents in the folder
UpdateGeometry(actions)

In [ ]:
# Analysis: This does some vortex panel method based abalysis of the wing (not useful for rocket)
#After running this you will have some new analysis documents in the folder
call(["vspaero", "-omp", "8", "WingBody_DegenGeom"])

In [ ]:
#This does some more heavy analysis for stability etc (not useful for rocket)
#After running this you will have some new analysis documents in the folder
call(["vspaero", "-omp", "8", "-stab", "WingBody_DegenGeom"])

In [ ]:
# Open analysis files
call(["vspviewer", "WingBody_DegenGeom"])

In [ ]:
#Open created OpenVSP document in OpenVSP
call(["vsp", "WingBody.vsp3"])


In [ ]:
#Open exported document in ParaView
call(["paraview", "WingBody.stl"])